# Pre-Requisite Installations

Please do the following steps initially:
- Install the AWS CLI from: https://aws.amazon.com/cli/
- Open a command line and type `aws configure`
- Leave the default region blank, and enter your AWS access id and secret key when prompted.

**(NOTE: Requires python >= 3.8)**

Dependencies (run the command in the next cell if you need these dependencies installed):
- boto3
- tqdm
- moviepy

In [15]:
# # Run these commands to install dependencies
# %pip install boto3
# %pip install tqdm
# %pip install moviepy
# %pip install wget (not needed)

# Import everything that is needed

In [16]:
# General imports
import os
from tqdm import tqdm
import json
import shutil
import random
import regex as re
import warnings
import uuid

# Video clipping
import datetime
from moviepy.editor import *
import subprocess

import tensorflow as tf

# Setup of ego4d downloads
from zipfile import ZipFile
from sys import platform
import requests
from io import BytesIO

# Custom classes
from utils import cd

# Print num of GPUs if available to use
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Download and Prepare Dataset

In [17]:
# Data paths that are defined
data_dir = 'data'
ego4d_setup_dir = 'ego4d_setup'
video_path = os.path.join(data_dir, 'videos')
annotation_path = os.path.join(data_dir, 'annotations')
training_path = os.path.join(video_path, 'training')
testing_path = os.path.join(video_path, 'testing')
validation_path = os.path.join(video_path, 'validation')

def perform_setup(delete_annotations=False, delete_clips=False, delete_setup=False, delete_data=False):
    # If you want to delete already existing folders and contents
    if delete_setup and os.path.exists(ego4d_setup_dir): shutil.rmtree(ego4d_setup_dir)
    if delete_clips and os.path.exists(video_path): shutil.rmtree(video_path)
    if delete_annotations and os.path.exists(annotation_path): shutil.rmtree(annotation_path)
    if delete_data and os.path.exists(data_dir): shutil.rmtree(data_dir)

    # If any dir doesn't exist, make it
    if not os.path.exists(ego4d_setup_dir): os.makedirs(ego4d_setup_dir)
    if not os.path.exists(annotation_path): os.makedirs(annotation_path)
    if not os.path.exists(video_path): os.makedirs(video_path)
    if not os.path.exists(training_path): os.makedirs(training_path)
    if not os.path.exists(testing_path): os.makedirs(testing_path)
    if not os.path.exists(validation_path): os.makedirs(validation_path)

# Uncomment to setup the dataset
# delete_annotations: resets data/annotations 
# delete_clips: resets data/videos
# delete_setup: resets ego4d_setup directory (should be deleted normally anyway)
# delete_data: deletes all of data directory and its subdirectories
perform_setup(delete_setup=True, delete_data=True)

> Retrieve ego4d data retrieval tool

In [18]:
# Retrieve the zip file for setup
print('Downloading the setup zip file..')

#Defining the zip file URL
url = 'https://github.com/facebookresearch/Ego4d/archive/refs/heads/main.zip'
# Split URL to get the file name
filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
req = requests.get(url)

# extracting the zip file contents
zipfile= ZipFile(BytesIO(req.content))
zipfile.extractall(ego4d_setup_dir)
print('Finished retrieving the setup files.')

Finished retrieving the setup files.


In [19]:
# Setting up the annotations and retrieving it
def setup_annotations(annotation_regex):
    # Check if annotations are already present
    if os.path.exists(annotation_path):
        files = next(os.walk(annotation_path), (None, None, []))[2]
        rgx = re.compile(annotation_regex)
        newlist = list(filter(rgx.match, files))

        # If not present, then download the annotations and save them
        if not newlist:
            print('Downloading annotations..')

            # Command to download ALL annotations
            command = ['python', '-m', 'ego4d.cli.cli', '--output_directory=ego4d_data', '--datasets', 'annotations', '--yes']
            with cd(os.path.join(ego4d_setup_dir, "Ego4d-main")):
                subprocess.run(command, check=True)
            # Copy and save only needed annotations in the data directory
            annotation_dir = os.path.join(ego4d_setup_dir, "Ego4d-main", "ego4d_data", "v1", "annotations")
            for file in os.listdir(annotation_dir):
                if "moments" in file: # TODO: Should be changed later to not have string
                    shutil.copy(os.path.join(annotation_dir, file), os.path.join(annotation_path))
            print('Finished downloading and setting up annotations.')
        else:
            print('Annotations already present!')
        
# Setting up the annotations
# annnotation_regex: regular expression of the files you want to copy from ALL annotations
setup_annotations("moments*")

Datasets to download: {'annotations'}
Download Path: ego4d_data/v1
Ego4D Metadata: ego4d_data/ego4d.json
Checking requested datasets and versions...
Created download directory for version 'v1' of dataset: 'annotations' at: ego4d_data/v1/annotations
Retrieving object metadata from S3...
Checking if latest file versions are already downloaded...


100%|██████████| 31/31 [00:01<00:00, 18.19file/s]
  0%|          | 0.00/0.02M [00:00<?, ?b/s]

No existing videos to filter.


100%|██████████| 0.02M/0.02M [01:32<00:00, 233Mb/s]    


Checking file integrity...
Finished downloading and setting up annotations.


# Pre-Process

### Process moments_train.json
> Process moments_train.json and filter videos with given classes.

In [30]:
# The classes to be used in this project
classes = ["wash_dishes_/_utensils_/_bakeware_etc.", "use_phone", "browse_through_clothing_items_on_rack_/_shelf_/_hanger"]
# Training, testing, and validation annotation files
# Filtered clip uids dictionary containing only classes listed above
filtered_dict = {
    'training': {
        'file_name': 'moments_train.json',
        'count': 15,
        'data': {}
    },
    'testing': {
        'file_name': 'moments_val.json',
        'count': 5,
        'data': {}
    },
    'validation': {
        'file_name': 'moments_val.json',
        'count': 5,
        'data': {}
    }
}

for data_type in filtered_dict:
    with open(os.path.join(annotation_path, filtered_dict[data_type]['file_name']), 'r') as json_file:
        # Load the annotation file
        loaded_json = json.load(json_file)

        # For each video and for each clips within the video
        # retrieve clip_uid, video_uids, label, start_time, and end_time
        for video in loaded_json['videos']:
            for clip in video['clips']:
                filtered_dict[data_type]['data'][clip["clip_uid"]] = {
                    "video_uid": video["video_uid"],
                    "annotations": {}
                }
                annotation = clip["annotations"][0]
                for label_item in annotation["labels"]:
                    label = label_item["label"]
                    if label in classes and (label_item["end_time"] - label_item["start_time"] >= 5):
                        if label in filtered_dict[data_type]['data'][clip["clip_uid"]]["annotations"]:
                            filtered_dict[data_type]['data'][clip["clip_uid"]]["annotations"][label].append({
                                "start_time": label_item["start_time"],
                                "end_time": label_item["end_time"]
                            })
                        else:
                            filtered_dict[data_type]['data'][clip["clip_uid"]]["annotations"][label] = [{
                                "start_time": label_item["start_time"],
                                "end_time": label_item["end_time"]
                            }]
                # Filter to make sure no clips with empty annotations are taken
                if not filtered_dict[data_type]['data'][clip["clip_uid"]]["annotations"]:
                    del filtered_dict[data_type]['data'][clip["clip_uid"]]

In [31]:
# print(len(filtered_dict))
# with open('test.json', 'w') as jp:
#     json.dump(filtered_dict, jp, indent=4)

### Trim videos into clips
> Using moviepy, split videos into clips

In [32]:
# Function to cut/trim the video to desired lengths and save it
def cut_video(video_path, times, save_path):
    # Load the video once and close using context manager
    with VideoFileClip(video_path) as vid_clip:
        for segment in times:
            # Segment information and random filename generation
            start_time = segment['start_time']
            end_time = segment['end_time']
            fileName = f"{uuid.uuid1()}.mp4"

            # Convert the amount of seconds in time xx:xx:xx.xxxxxx
            start_time = str(datetime.timedelta(seconds = start_time))
            end_time = str(datetime.timedelta(seconds = end_time))

            # Clip the video and save it to the specified path
            clip = vid_clip.subclip(start_time, end_time)
            if not os.path.exists(save_path): os.makedirs(save_path)
            clip.write_videofile(os.path.join(save_path, fileName))

In [34]:
%%capture cap --no-stderr
# ^ To save the output to external file because it's waaaaay too large to display in notebook
# Function to retrieve a sample of clips for each class
def generate_clip_uids(filtered_annotations):
    # Seperate clip uids by their class
    all_clips_dict = {
        'training': {},
        'testing': {},
        'validation': {}
    }
    for clip_type in all_clips_dict:
        for clip_uid, clip_info in filtered_annotations[clip_type]['data'].items():
            for annotation in clip_info["annotations"]:
                if annotation in all_clips_dict[clip_type]:
                    all_clips_dict[clip_type][annotation].append(clip_uid)
                else:
                    all_clips_dict[clip_type][annotation] = [clip_uid]
        # Grab random sample of num_clips for each class
        for annotation in all_clips_dict[clip_type]:
            all_clips_dict[clip_type][annotation] = random.sample(all_clips_dict[clip_type][annotation], filtered_annotations[clip_type]['count'])
    return all_clips_dict


# Function to download and setup the clips 
def setup_clips(filtered_annotations, save_path, overwrite=False):
    # Check if the clips already exist if we are not overwriting
    if not overwrite:
        dirs = next(os.walk(video_path), (None, None, []))[1]
        for dir in dirs:
            subdirs = next(os.walk(video_path, dir), (None, None, []))[2]
            for subdir in subdirs:
                files = next(os.walk(os.path.join(video_path, dir, subdir)), (None, None, []))[2]
                if dir.replace(";", "/") in all_clips_dict['training'] and files:
                    warnings.warn("Clips may already exist. If you want to redo the process, set the `overwrite` argument to true while calling the function.")
                    return

    # Get all the randomly generated clip samples and go through them
    all_clips_dict = generate_clip_uids(filtered_annotations)

    # Go through each type of data (training, testing, validation)
    for data_type in filtered_annotations:
        for cls, clip_uids in all_clips_dict[data_type].items():
            useable_name = cls.replace("/", ";") # Paths can't have / in their name so, replace it

            # Make sure the required path exists without old items
            if os.path.exists(os.path.join(video_path, data_type, useable_name)): shutil.rmtree(os.path.join(video_path, data_type, useable_name))
            os.makedirs(os.path.join(video_path, data_type, useable_name))

            print(f"Processing clips for class: {cls}")
            for clip_uid in clip_uids:
                # Command to retrieve clip by its uid
                command = ['python', '-m', 'ego4d.cli.cli', '--output_directory=temp_clip', 
                '--datasets', 'clips', '--video_uids', clip_uid, '--yes']
                # Path to the downloaded clip
                full_vid_path = os.path.join(ego4d_setup_dir, "Ego4d-main", "temp_clip", "v1", "clips")
                
                # Use context manager to ensure any change in directory is returned to original
                with cd(os.path.join(ego4d_setup_dir, "Ego4d-main")):
                    # Download the video
                    subprocess.run(command, check=True)
                    
                # Get the clip information and trim the video
                clip_info = filtered_annotations[data_type]['data'][clip_uid]
                for label in clip_info['annotations']:
                    folder_label = label.replace("/", ";")
                    cut_video(os.path.join(full_vid_path, f"{clip_uid}.mp4"), clip_info['annotations'][label], os.path.join(video_path, data_type, folder_label))
                
                # Clean up after for the specific clip
                shutil.rmtree(os.path.join(ego4d_setup_dir, "Ego4d-main", "temp_clip"))
    
    # Clean up the overall setup directory
    shutil.rmtree(ego4d_setup_dir)

# Download and setup clips
# filtered_dict: previously filtered dictionary of clips
# video_path: path to save the clips
setup_clips(filtered_dict, video_path)

Datasets to download: {'clips'}
Download Path: temp_clip/v1
Ego4D Metadata: temp_clip/ego4d.json
Checking requested datasets and versions...
Created download directory for version 'v1' of dataset: 'clips' at: temp_clip/v1/clips
Only downloading a subset of the video files because the 'video_uids' flag has been set on the command line or in the config file. A total of 1 video files will be downloaded.

Retrieving object metadata from S3...
Checking if latest file versions are already downloaded...


100%|██████████| 1/1 [00:00<00:00,  4.94file/s]
  0%|          | 0.00/666k [00:00<?, ?b/s]

No existing videos to filter.


100%|██████████| 666k/666k [00:08<00:00, 80.7Mb/s] 


KeyError: '5123a4e0-afd2-4e6b-8414-161fbdb71d6f'

Checking file integrity...


# Load the dataset

## Data Visualization

In [ ]:
# fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(15, 15))

# # flatten the axis into a 1-d array to make it easier to access each axes
# axs = axs.flatten()

# # iterate through and enumerate the files, use i to index the axes
# for i in range(3):
#     sample = train_dataset[i]

#     # add the image to the axes
#     axs[i].imshow(sample['image'])
#     axs[i].axis('off')

# # add a figure title
# fig.suptitle('Visualizing Training Dataset', fontsize=18)

# fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(15, 15))

# # flatten the axis into a 1-d array to make it easier to access each axes
# axs = axs.flatten()

# # iterate through and enumerate the files, use i to index the axes
# for i in range(3):
#     sample = test_dataset[i]

#     # add the image to the axes
#     axs[i].imshow(sample['image'])
#     axs[i].axis('off')

# # add a figure title
# fig.suptitle('Visualizing Testing Dataset', fontsize=18)